In [44]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
import numpy as np
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.metrics import confusion_matrix


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wangtao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
# df = pd.read_excel('FB2000_one_hot_post.xlsx')
df = pd.read_excel('FB2000_one_hot_post.xlsx')
df.shape

(1968, 13)

In [46]:
df.head()

,text,covid,economic,education,environmental,foreign policy,governance,health,immigration,military,safety,social cultural,social programs
0,last quarter bernie sanders pete buttigieg and...,0,0,0,0,0,0,0,0,0,0,0,0
1,every one of the democratic candidates remaini...,0,0,0,0,0,0,0,0,0,0,0,0
2,a young woman came up to me after my speech in...,0,0,1,0,0,0,0,0,0,0,0,1
3,como presidenta elizabeth proporcionarн_a el m...,0,0,0,0,0,0,0,0,0,0,0,1
4,una cosa queda clara ahorrar para una jubilaci...,0,0,0,0,0,0,0,0,0,0,0,1


In [47]:
categories = list(df.columns.values)
categories = categories[1:]
print(categories)

['covid', 'economic', 'education', 'environmental', 'foreign policy', 'governance', 'health', 'immigration', 'military', 'safety', 'social cultural', 'social programs']


In [48]:
df = df.dropna(subset = ['text'])
df.shape

(1962, 13)

In [49]:
#no of comments in each category
counts = []
for category in categories:
    counts.append((category, df[category].sum()))
df_stats = pd.DataFrame(counts, columns=['category', 'number of comments'])
print(df_stats)

           category  number of comments
0             covid                  40
1          economic                 386
2         education                 122
3     environmental                 131
4    foreign policy                  70
5        governance                 190
6            health                 214
7       immigration                  90
8          military                  50
9            safety                 164
10  social cultural                 253
11  social programs                 181


In [50]:
train = df.text
vect = TfidfVectorizer(stop_words = stop_words,ngram_range=(2, 3),min_df = 2)
vect.fit(train)
X_trans = vect.transform(train)
X_trans

<1962x6229 sparse matrix of type '<class 'numpy.float64'>'
	with 18281 stored elements in Compressed Sparse Row format>

In [51]:
import json 
# with open ('2019_public_comment.json','w') as f1:
#     json.dump(dic,f1)
# with open ('2020_may30_lexiconV1.json') as f1:
#     dic = json.load(f1)
# with open ('2020_june_lexiconV3_4.json') as f1:
with open ('2020_june_lexiconV4_5.json') as f1:
    dic = json.load(f1)

In [52]:
unigramtopic_dict = {}
ngramtopic_dict = {}

for key, value in dic.items():
    ngramlist=[]
    unigramlist=[]
    for v in value:
        if ' ' in v or '-' in v or '/' in v:
            ngramlist.append(v)
        if not ' ' in v and not '-' in v and not '/' in v :
            unigramlist.append(v)
    
    unigramtopic_dict[key] = unigramlist
    ngramtopic_dict[key] = ngramlist

# unigramtopic_dict

In [53]:
def plot_coefficients(classifier, feature_names, top_features=20):
    coef = classifier.coef_.ravel()
    # print(coef[0])
    # print(sorted(coef)[-20:])
    top_positive_coefficients = np.argsort(coef)[-top_features:] # argsort return the index of the features
 #print(top_positive_coefficients)
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
 # create plot
    #plt.figure(figsize=(15, 5))
    #colors = ['red'if c < 0 else 'blue' for c in coef[top_coefficients]]
    #plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
    feature_names = np.array(feature_names)
    # print(top_positive_coefficients)
    return (feature_names[top_positive_coefficients])
 
    
#     plt.xticks(np.arange(1, 1 + 2 * top_features), feature_names[top_coefficients], rotation=60, ha='right')
#     plt.show()

In [56]:
orc = OneVsRestClassifier(LinearSVC(max_iter=1000),n_jobs=1)

for category in categories:
    orc.fit(X_trans, df[category])
    prediction = cross_val_predict(orc, X_trans, df[category],cv=5)
   #print(prediction.mean())
   #prediction = orc.predict(X_test_trans)
    print('\n')
    print(category)
   #print('accuracy is {}'.format(accuracy_score(df[category], prediction)))
    p,r,f1,x = precision_recall_fscore_support(df[category], prediction, average='macro')
    print("precision =",p,"Recall = ",r,"F1 =", f1) 
    tn,fp,fn,tp = confusion_matrix(df[category],prediction).ravel()
    print("True Negative =",tn,"False Positive =",fp,"False Negative =", fn,"True Poistive = ",tp)
    top_positives = plot_coefficients(orc, vect.get_feature_names())
    if category: #!= 'governa nce':
        l = []
        for word in top_positives:
            if word not in unigramtopic_dict[category] and word not in ngramtopic_dict[category]:
                l.append(word)
        print(len(l))
        print(l)
        





covid
precision = 0.7405419222903886 Recall =  0.5367195629552549 F1 = 0.5600600937282777
True Negative = 1919 False Positive = 3 False Negative = 37 True Poistive =  3
18
['invisible enemy', 'progress made', 'public health', 'congress act', 'crisis donald', 'crisis donald trump', 'much better', '40 000', 'protect working', '10 months', 'days new', 'administration failure', 'country world', 'front lines crisis', 'lines crisis', 'two months', 'could make', 'health officials']


economic
precision = 0.8199394649401112 Recall =  0.6261490360590201 F1 = 0.6558127282605672
True Negative = 1549 False Positive = 27 False Negative = 282 True Poistive =  104
15
['democrat party', 'tax cuts', 'help people', 'got plan', 'stock market', 'last year', 'teachers value', 'transition new', 'unemployment rate', 'american families', 'economy works', 'working people', 'minimum wage', 'working families', 'wall street']


education
precision = 0.9600491137802576 Recall =  0.6226790805416964 F1 = 0.6837474

## extract social programs

In [57]:
# X_trans the vectorized text

orc.fit(X_trans,df['social programs'])

OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                        fit_intercept=True, intercept_scaling=1,
                                        loss='squared_hinge', max_iter=1000,
                                        multi_class='ovr', penalty='l2',
                                        random_state=None, tol=0.0001,
                                        verbose=0),
                    n_jobs=1)

In [58]:
prediction = cross_val_predict(orc, X_trans, df[category],cv=5)

In [59]:
p,r,f1,x = precision_recall_fscore_support(df[category], prediction, average='macro')
print("precision =",p,"Recall = ",r,"F1 =", f1)
tn,fp,fn,tp = confusion_matrix(df[category],prediction).ravel()
print("True Negative =",tn,"False Positive =",fp,"False Negative =", fn,"True Poistive = ",tp)

precision = 0.8878017409678678 Recall =  0.6571328417519489 F1 = 0.7137637452367991
True Negative = 1770 False Positive = 11 False Negative = 123 True Poistive =  58


In [60]:
top_positives = plot_coefficients(orc, vect.get_feature_names(),30)
l=[]
for word in top_positives:
    if word not in unigramtopic_dict['social programs'] and word not in ngramtopic_dict['social programs']:
        l.append(word)
print(l)

['today tomorrow', 'right privilege', 'playing field', 'need medicare', 'care act', 'medical bills', 'wealthy corporations', 'universal healthcare', 'medical care', 'decent housing', 'billion hbcus', 'left right', 'family leave', 'public schools', 'insurance companies', 'high quality', 'mental health']
